In [ ]:
! pip install imdbPY

In [ ]:
import imdb
import csv
import pandas as pd
from imdb import IMDb, IMDbDataAccessError

In [ ]:
ia = imdb.IMDb()
csv_file = '/kaggle/input/imdbdata/imdb_data.csv'

In [ ]:
data=pd.read_csv(r'/kaggle/input/movielens/movies.csv')

In [ ]:
#old verison 
import imdb
import csv
import random
import pandas as pd
import concurrent.futures

# Load the movie data
imdbs = data['imdbId']

# Set up the IMDb API
ia = imdb.IMDb()

# Define the batch size for processing
batch_size = 100

# Define the function to get movie data
def get_movie_data(movie_id):
    try:
        movie = ia.get_movie(movie_id)
        ia.update(movie, 'reviews')
        ia.update(movie, 'cast')
        data = {
            'film_name': movie.get('title', ''),
            'film_rate': movie.get('rating', 0.0),  # default to 0.0 if no rating
            'plot': movie.get('plot', ''),
            'cover_url': movie.get('cover url', ''),
            'cast': ', '.join([person['name'] for person in movie.get('cast', [])[:3]]) if movie.get('cast') else '',  # return empty string if cast is not available
            'reviews': movie.get('reviews', [{}])[0].get('content', '')  # default to empty string if no reviews
        }
        return data
    except IMDbDataAccessError as e:
        return {'film_name': '', 'film_rate': 0.0, 'plot': '', 'cover_url': '', 'cast': '', 'reviews': ''}  # return default values
    except urllib.error.HTTPError as e:
        return {'film_name': '', 'film_rate': 0.0, 'plot': '', 'cover_url': '', 'cast': '', 'reviews': ''}  # return default values

# Define the function to process a batch of movie IDs
def process_batch(movie_ids):
    data_list = []
    for movie_id in movie_ids:
        data_list.append(get_movie_data(movie_id))
    return data_list

# Create a list to store the processed data
data_list = []

# Create a ThreadPoolExecutor to parallelize the API calls
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    # Split the movie IDs into batches
    batches = [imdbs[i:i + batch_size] for i in range(0, len(imdbs), batch_size)]
    
    # Process each batch in parallel
    futures = [executor.submit(process_batch, batch) for batch in batches]
    
    # Collect the results
    for future in concurrent.futures.as_completed(futures):
        data_list.extend(future.result())
csv_file='/kaggle/input/imdbdata/imdb_data.csv'
# Save the processed data to a CSV file
with open(csv_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['film_name', 'film_rate', 'plot', 'cover_url', 'cast', 'reviews'])
    for data in data_list:
        writer.writerow(list(data.values()))

In [ ]:
#updated version with more features 
import imdb
import csv
import random
import pandas as pd
import concurrent.futures

# Load the movie data
imdbs = data['imdbId']

# Set up the IMDb API
ia = imdb.IMDb()

# Define the batch size for processing
batch_size = 100

# Define the function to get movie data
def get_movie_data(movie_id):
    try:
        movie = ia.get_movie(movie_id)
        ia.update(movie, 'reviews')
        ia.update(movie, 'cast')
        ia.update(movie, 'directors')
        ia.update(movie, 'writers')

        data = {
            'imdbId': movie_id,
            'film_name': movie.get('title', ''),
            'film_rate': movie.get('rating', 0.0),  # default to 0.0 if no rating
            'plot': movie.get('plot', [''])[0],  # take the first plot synopsis
            'cover_url': movie.get('cover url', ''),
            'cast': ', '.join([person['name'] for person in movie.get('cast', [])[:3]]) if movie.get('cast') else '',  # top 3 cast
            'reviews': movie.get('reviews', [{}])[0].get('content', ''),  # first review content
            'director': ', '.join([person['name'] for person in movie.get('directors', [])]) if movie.get('directors') else '',
            'writer': ', '.join([person['name'] for person in movie.get('writers', [])]) if movie.get('writers') else '',
            'genre': ', '.join(movie.get('genres', [])) if movie.get('genres') else ''  # join genre list
        }
        return data
    except imdb.IMDbDataAccessError as e:
        return {'imdbId': movie_id, 'film_name': '', 'film_rate': 0.0, 'plot': '', 'cover_url': '', 'cast': '', 'reviews': '', 'director': '', 'writer': '', 'genre': ''}
    except urllib.error.HTTPError as e:
        return {'imdbId': movie_id, 'film_name': '', 'film_rate': 0.0, 'plot': '', 'cover_url': '', 'cast': '', 'reviews': '', 'director': '', 'writer': '', 'genre': ''}

# Define the function to process a batch of movie IDs
def process_batch(movie_ids):
    data_list = []
    for movie_id in movie_ids:
        data_list.append(get_movie_data(movie_id))
    return data_list

# Create a list to store the processed data
data_list = []

# Create a ThreadPoolExecutor to parallelize the API calls
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    # Split the movie IDs into batches
    batches = [imdbs[i:i + batch_size] for i in range(0, len(imdbs), batch_size)]
    
    # Process each batch in parallel
    futures = [executor.submit(process_batch, batch) for batch in batches]
    
    # Collect the results
    for future in concurrent.futures.as_completed(futures):
        data_list.extend(future.result())

# Specify the path to save the CSV
csv_file = '/kaggle/input/imdbdata/imdb_data.csv'

# Save the processed data to a CSV file
with open(csv_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['imdbId', 'film_name', 'film_rate', 'plot', 'cover_url', 'cast', 'reviews', 'director', 'writer', 'genre'])
    for data in data_list:
        writer.writerow([data['imdbId'], data['film_name'], data['film_rate'], data['plot'], data['cover_url'], data['cast'], data['reviews'], data['director'], data['writer'], data['genre']])


In [ ]:
import imdb
import pandas as pd
import concurrent.futures

# Extract the list of IMDb IDs
imdbs = data['imdbId']

# Set up the IMDb API
ia = imdb.IMDb()

# Define the batch size for processing
batch_size = 100

# Define the function to get movie data
def get_movie_data(movie_id):
    try:
        movie = ia.get_movie(movie_id)
        ia.update(movie, 'reviews')
        ia.update(movie, 'cast')
        ia.update(movie, 'directors')
        ia.update(movie, 'writers')

        # Return the new data, using `.get()` to safely handle missing fields
        data = {
            'imdbId': movie_id,
            'director': ', '.join([person.get('name', '') for person in movie.get('directors', [])]) if movie.get('directors') else '',
            'writer': ', '.join([person.get('name', '') for person in movie.get('writers', [])]) if movie.get('writers') else '',
            'genre': ', '.join(movie.get('genres', [])) if movie.get('genres') else ''  # join genre list
        }
        return data
    except imdb.IMDbDataAccessError as e:
        return {'imdbId': movie_id, 'director': '', 'writer': '', 'genre': ''}
    
# Define the function to process a batch of movie IDs
def process_batch(movie_ids):
    data_list = []
    for movie_id in movie_ids:
        data_list.append(get_movie_data(movie_id))
    return data_list

# Create a list to store the processed data
data_list = []

# Create a ThreadPoolExecutor to parallelize the API calls
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    # Split the movie IDs into batches
    batches = [imdbs[i:i + batch_size] for i in range(0, len(imdbs), batch_size)]
    
    # Process each batch in parallel
    futures = [executor.submit(process_batch, batch) for batch in batches]
    
    # Collect the results
    for future in concurrent.futures.as_completed(futures):
        data_list.extend(future.result())

# Convert the collected data into a DataFrame
new_data_df = pd.DataFrame(data_list)

# Merge the new data with the existing dataset
updated_data = pd.merge(data, new_data_df, on='imdbId', how='left')

# Save the updated data back to the CSV file
updated_data.to_csv(csv_file, index=False)


In [ ]:
data_list

In [ ]:
import pandas as pd

# Create a Pandas DataFrame from the data_list
df = pd.DataFrame(data_list)

df.to_csv('imdb_data.csv', index=False)
from IPython.display import FileLink

# This will create a download link for the file
FileLink('imdb_data.csv')

# assosication

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth, association_rules

# Load the dataset in chunks
chunksize = 1000
chunks = pd.read_csv(r'/kaggle/input/rating/ratings.csv', chunksize=chunksize)

# Concatenate all chunks into a single DataFrame
ratings = pd.concat(chunks, ignore_index=True)

# Filter only ratings of 4.0 and above for "liked" movies
ratings_filtered = ratings[ratings['rating'] >= 4.0]

# Create a user-item matrix
user_movie_matrix = ratings_filtered.pivot_table(index='userId', columns='movieId', aggfunc='size', fill_value=0)

# Apply FP-Growth to find frequent item sets
frequent_itemsets = fpgrowth(user_movie_matrix, min_support=0.02, use_colnames=True)

# Generate association rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

# Sort the rules by lift
rules = rules.sort_values(by='lift', ascending=False)

# Display the rules
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])


# assosiation model 

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth, association_rules

# Load the dataset in chunks
chunksize = 1000
chunks = pd.read_csv(r'/kaggle/input/rating/ratings.csv', chunksize=chunksize)

# Concatenate all chunks into a single DataFrame
ratings = pd.concat(chunks, ignore_index=True)

# Filter only ratings of 4.0 and above for "liked" movies
ratings_filtered = ratings[ratings['rating'] >= 4.0]

# Create a user-item matrix, where 1 represents a "liked" movie (rating 4.0 or above)
user_movie_matrix = ratings_filtered.pivot_table(index='userId', columns='movieId', aggfunc='size', fill_value=0)

# Apply FP-Growth to find frequent item sets
frequent_itemsets = fpgrowth(user_movie_matrix, min_support=0.02, use_colnames=True)

# Generate association rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

# Sort the rules by lift
rules = rules.sort_values(by='lift', ascending=False)

# Load the movies dataset to map movie IDs to movie titles
movies = pd.read_csv(r'/kaggle/input/movielens/movies.csv')

# Create a function to get movie recommendations
def get_movie_id(movie_title):
    """
    Get the movieId for the given movie title from the movies dataset.
    """
    movie_id = movies[movies['title'].str.contains(movie_title, case=False, na=False)]['movieId']
    if len(movie_id) > 0:
        return movie_id.values[0]
    else:
        return None

def recommend_movies(movie_title, rules, movies, top_n=3):
    """
    Recommend associated movies based on the input movie title.
    """
    movie_id = get_movie_id(movie_title)
    if movie_id is None:
        return f"Movie '{movie_title}' not found in the dataset."

    # Find rules where the input movie is in the antecedents
    matching_rules = rules[rules['antecedents'].apply(lambda x: movie_id in x)]

    if matching_rules.empty:
        return f"No associated movies found for '{movie_title}'."

    # Extract the consequents from the association rules
    movie_recommendations = []
    for _, row in matching_rules.iterrows():
        for consequent in row['consequents']:
            if consequent != movie_id:
                movie_recommendations.append((consequent, row['lift']))

    # Remove duplicates and sort by lift value (descending)
    movie_recommendations = sorted(list(set(movie_recommendations)), key=lambda x: x[1], reverse=True)

    # Get the top N movie recommendations
    recommended_movie_ids = [rec[0] for rec in movie_recommendations[:top_n]]
    recommended_movies = movies[movies['movieId'].isin(recommended_movie_ids)]['title'].values

    return recommended_movies

# Example usage:
movie_name = 'Interstellar'  # Change this to the movie title you want recommendations for
recommended_movies = recommend_movies(movie_name, rules, movies)

if isinstance(recommended_movies, str):
    print(recommended_movies)  # If the function returns an error message
else:
    print(f"Movies recommended based on '{movie_name}':")
    for i, movie in enumerate(recommended_movies, 1):
        print(f"{i}. {movie}")


# assosiation model low resources

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth, association_rules
from scipy.sparse import csr_matrix

# Load the dataset in chunks to avoid memory overload
chunksize = 10**6  # Increase the chunksize to process more data at once but keep it manageable
chunks = pd.read_csv(r'/kaggle/input/rating/ratings.csv', chunksize=chunksize, usecols=['userId', 'movieId', 'rating'])

# Initialize an empty list to store filtered data
filtered_chunks = []

# Process each chunk individually
for chunk in chunks:
    # Filter ratings of 4.0 and above for "liked" movies
    filtered_chunk = chunk[chunk['rating'] >= 4.0]
    filtered_chunks.append(filtered_chunk)

# Concatenate all filtered chunks into a single DataFrame
ratings_filtered = pd.concat(filtered_chunks, ignore_index=True)

# Create a user-item matrix in sparse format to save memory
user_movie_matrix = ratings_filtered.pivot_table(index='userId', columns='movieId', aggfunc='size', fill_value=0)
user_movie_matrix_sparse = csr_matrix(user_movie_matrix.values)

# Apply FP-Growth to find frequent item sets
frequent_itemsets = fpgrowth(pd.DataFrame(user_movie_matrix_sparse.todense(), columns=user_movie_matrix.columns), min_support=0.02, use_colnames=True)

# Generate association rules from the frequent item sets
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

# Sort the rules by lift to find the most interesting ones
rules = rules.sort_values(by='lift', ascending=False)

# Load the movies dataset to map movie IDs to movie titles
movies = pd.read_csv(r'/kaggle/input/movielens/movies.csv')

# Create a function to get movie recommendations
def get_movie_id(movie_title):
    """
    Get the movieId for the given movie title from the movies dataset.
    """
    movie_id = movies[movies['title'].str.contains(movie_title, case=False, na=False)]['movieId']
    if len(movie_id) > 0:
        return movie_id.values[0]
    else:
        return None

def recommend_movies(movie_title, rules, movies, top_n=3):
    """
    Recommend associated movies based on the input movie title.
    """
    movie_id = get_movie_id(movie_title)
    if movie_id is None:
        return f"Movie '{movie_title}' not found in the dataset."

    # Find rules where the input movie is in the antecedents
    matching_rules = rules[rules['antecedents'].apply(lambda x: movie_id in x)]

    if matching_rules.empty:
        return f"No associated movies found for '{movie_title}'."

    # Extract the consequents from the association rules
    movie_recommendations = []
    for _, row in matching_rules.iterrows():
        for consequent in row['consequents']:
            if consequent != movie_id:
                movie_recommendations.append((consequent, row['lift']))

    # Remove duplicates and sort by lift value (descending)
    movie_recommendations = sorted(list(set(movie_recommendations)), key=lambda x: x[1], reverse=True)

    # Get the top N movie recommendations
    recommended_movie_ids = [rec[0] for rec in movie_recommendations[:top_n]]
    recommended_movies = movies[movies['movieId'].isin(recommended_movie_ids)]['title'].values

    return recommended_movies

# Example usage:
movie_name = 'Interstellar'  # Change this to the movie title you want recommendations for
recommended_movies = recommend_movies(movie_name, rules, movies)

if isinstance(recommended_movies, str):
    print(recommended_movies)  # If the function returns an error message
else:
    print(f"Movies recommended based on '{movie_name}':")
    for i, movie in enumerate(recommended_movies, 1):
        print(f"{i}. {movie}")


# final try 

In [2]:
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth, association_rules
from scipy.sparse import csr_matrix
import re

# Function to clean movie titles by removing special characters and lowercasing
def clean_movie_title(title):
    # Remove anything in parentheses (like year)
    title = re.sub(r'\(.*?\)', '', title)
    # Remove extra spaces and lowercase the title
    title = re.sub(r'\s+', ' ', title).strip().lower()
    return title

# Load the dataset in chunks
chunksize = 10**6
chunks = pd.read_csv(r'/kaggle/input/rating/ratings.csv', chunksize=chunksize, usecols=['userId', 'movieId', 'rating'])

# Initialize an empty list to store filtered data
filtered_chunks = []

# Process each chunk individually
for chunk in chunks:
    # Filter ratings of 4.0 and above for "liked" movies
    filtered_chunk = chunk[chunk['rating'] >= 4.0]
    filtered_chunks.append(filtered_chunk)

# Concatenate filtered chunks
ratings_filtered = pd.concat(filtered_chunks, ignore_index=True)

# Create a user-item matrix in sparse format
user_movie_matrix = ratings_filtered.pivot_table(index='userId', columns='movieId', aggfunc='size', fill_value=0)
user_movie_matrix_sparse = csr_matrix(user_movie_matrix.values)

# Apply FP-Growth to find frequent item sets
frequent_itemsets = fpgrowth(pd.DataFrame(user_movie_matrix_sparse.todense(), columns=user_movie_matrix.columns), min_support=0.05, use_colnames=True)

# Generate association rules from frequent item sets
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

# Sort rules by lift
rules = rules.sort_values(by='lift', ascending=False)

# Load the movies dataset
movies = pd.read_csv(r'/kaggle/input/movielens/movies.csv')

# Apply the clean_movie_title function to standardize movie titles
movies['clean_title'] = movies['title'].apply(clean_movie_title)

# Create a function to get movie recommendations
def get_movie_id(movie_title):
    """
    Get the movieId for the given movie title from the movies dataset.
    Clean the input movie title for consistency.
    """
    cleaned_title = clean_movie_title(movie_title)
    movie_id = movies[movies['clean_title'] == cleaned_title]['movieId']
    
    if len(movie_id) > 0:
        return movie_id.values[0]
    else:
        return None

def recommend_movies(movie_title, rules, movies, top_n=3):
    """
    Recommend associated movies based on the input movie title.
    """
    movie_id = get_movie_id(movie_title)
    if movie_id is None:
        return f"Movie '{movie_title}' not found in the dataset."

    # Find rules where the input movie is in the antecedents
    matching_rules = rules[rules['antecedents'].apply(lambda x: movie_id in x)]

    if matching_rules.empty:
        return f"No associated movies found for '{movie_title}'."

    # Extract the consequents from the association rules
    movie_recommendations = []
    for _, row in matching_rules.iterrows():
        for consequent in row['consequents']:
            if consequent != movie_id:
                movie_recommendations.append((consequent, row['lift']))

    # Remove duplicates and sort by lift value
    movie_recommendations = sorted(list(set(movie_recommendations)), key=lambda x: x[1], reverse=True)

    # Get top N movie recommendations
    recommended_movie_ids = [rec[0] for rec in movie_recommendations[:top_n]]
    recommended_movies = movies[movies['movieId'].isin(recommended_movie_ids)]['title'].values

    return recommended_movies



/opt/conda/lib/python3.10/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


Movie 'Shawshank Redemption' not found in the dataset.


In [12]:
# Example usage:
movie_name = 'Heat (1995)'  # You can try different variations like 'interstellar'
recommended_movies = recommend_movies(movie_name, rules, movies)

if isinstance(recommended_movies, str):
    print(recommended_movies)  # Error message
else:
    print(f"Movies recommended based on '{movie_name}':")
    for i, movie in enumerate(recommended_movies, 1):
        print(f"{i}. {movie}")

Movies recommended based on 'Heat (1995)':
1. Silence of the Lambs, The (1991)
2. Saving Private Ryan (1998)
